In [18]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import xarray as xr
from os.path import exists, join
import os

%matplotlib inline

In [19]:
preds = np.load("./pred_arrays.npy")
true = np.load("./true_arrays.npy")

In [20]:
lat_lon_weights = xr.open_dataset(
    "/glade/u/home/wchapman/MLWPS/DataLoader/static_variables_ERA5_zhght.nc"
)

In [21]:
means = xr.open_dataset(
    "/glade/campaign/cisl/aiml/wchapman/MLWPS/STAGING/All_2010_staged.SLO.mean.nc"
)
sds = xr.open_dataset(
    "/glade/campaign/cisl/aiml/wchapman/MLWPS/STAGING/All_2010_staged.SLO.std.nc"
)

In [22]:
means["Q"].values
sds["Q"].values

array(0.00379492, dtype=float32)

Plot un-normalized data on global map

In [24]:
out_dir = "pred_images"
if not exists(out_dir):
    os.makedirs(out_dir)
for t in range(24):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.EckertIII())
    ax.set_global()
    ax.coastlines("110m", alpha=0.5)

    pout = ax.pcolormesh(
        lat_lon_weights["longitude"],
        lat_lon_weights["latitude"],
        (preds[t] * sds["Q"].values + means["Q"].values) * 1000,
        transform=ccrs.PlateCarree(),
        vmin=0,
        vmax=20,
        cmap="RdBu",
    )
    plt.colorbar(pout, ax=ax, orientation="horizontal", fraction=0.05, pad=0.01)
    plt.title(f"Q (g/kg) F{t:02d}")
    plt.savefig(
        join(f"{out_dir}", f"global_q_{t:02d}.png"), dpi=300, bbox_inches="tight"
    )
    plt.close()

Create animated GIF with image magick

In [25]:
! convert   -delay 20   -loop 0 pred_images/global_q_*.png global_q_fixed.gif